# ETL (Extract, Transform, Load) para Análisis de Luminarias Urbanas

## 📋 Introducción

Este notebook implementa un proceso **ETL completo** para el análisis de luminarias urbanas del proyecto ECOLUZ. El proceso integra tanto análisis **supervisado** como **no supervisado** para el mantenimiento predictivo de luminarias.

### 🎯 Objetivos del ETL

1. **Extraer** datos de MongoDB de luminarias urbanas
2. **Transformar** los datos aplicando limpieza, normalización y ingeniería de características
3. **Cargar** los datos procesados para análisis supervisado y no supervisado
4. **Generar insights** mediante visualizaciones y modelos predictivos

### 🔄 Flujo del Proceso

```
MongoDB → Extract → Clean → Feature Engineering → Supervised Analysis → Unsupervised Analysis → Insights
```

### 📊 Metodologías Implementadas

- **Análisis Supervisado**: Random Forest para clasificación del estado de luminarias
- **Análisis No Supervisado**: K-Means clustering y PCA para identificación de patrones
- **Visualización Avanzada**: Gráficos de importancia, matrices de confusión y clustering

## 📚 Marco Teórico

### ETL en el Contexto de Machine Learning

**ETL (Extract, Transform, Load)** es fundamental para el análisis de datos de calidad:

- **Extract**: Extracción de datos desde fuentes heterogéneas (MongoDB, APIs, archivos)
- **Transform**: Limpieza, normalización, agregación y ingeniería de características
- **Load**: Carga de datos limpios en estructuras optimizadas para análisis

### Análisis Supervisado vs No Supervisado

#### 🎯 **Análisis Supervisado**
- **Objetivo**: Predecir el estado de luminarias (bueno, regular, malo)
- **Algoritmo**: Random Forest Classifier
- **Ventajas**: 
  - Maneja variables mixtas (numéricas y categóricas)
  - Proporciona importancia de variables
  - Robusto ante outliers
- **Métricas**: Accuracy, Precision, Recall, F1-Score

#### 🔍 **Análisis No Supervisado**
- **Objetivo**: Descubrir patrones ocultos y agrupamientos naturales
- **Algoritmos**: K-Means clustering, PCA (Principal Component Analysis)
- **Ventajas**:
  - No requiere etiquetas predefinidas
  - Identifica estructuras latentes en los datos
  - Útil para segmentación y reducción dimensional
- **Métricas**: Silhouette Score, Within-Cluster Sum of Squares (WCSS)

### Técnicas de Visualización

1. **Importancia de Variables**: Identifica características más relevantes
2. **Matriz de Confusión**: Evalúa rendimiento de clasificación
3. **PCA 2D**: Visualiza clusters en espacio reducido
4. **Método del Codo**: Determina número óptimo de clusters

## 📦 Importación de Librerías

Importamos todas las librerías necesarias para el proceso ETL completo:

In [ ]:
# ==========================================
# LIBRERÍAS PARA PROCESAMIENTO DE DATOS
# ==========================================
import pandas as pd
import numpy as np
import os
from math import pi

# ==========================================
# CONEXIÓN A BASE DE DATOS
# ==========================================
from dotenv import load_dotenv
from pymongo import MongoClient

# ==========================================
# VISUALIZACIÓN
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# ==========================================
# MACHINE LEARNING - SUPERVISADO
# ==========================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# ==========================================
# MACHINE LEARNING - NO SUPERVISADO
# ==========================================
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# ==========================================
# ANÁLISIS ESTADÍSTICO
# ==========================================
from scipy import stats

print("✅ Todas las librerías importadas correctamente")

## 🔌 FASE 1: EXTRACT (Extracción)

### Configuración de Conexión a MongoDB

In [ ]:
# Cargar variables de entorno
load_dotenv()
MONGO_URI = os.getenv("CONECTION_DB")

# Validar conexión
if MONGO_URI:
    print("✅ URI de MongoDB cargada correctamente")
    print(f"🔗 Conectando a: {MONGO_URI[:20]}...")
else:
    print("❌ No se encontró la URI de MongoDB")
    print("💡 Asegúrate de tener un archivo .env con CONECTION_DB")

# Test de conexión
try:
    client = MongoClient(MONGO_URI)
    client.admin.command('ping')
    print("✅ Conexión a MongoDB exitosa")
    client.close()
except Exception as e:
    print(f"❌ Error de conexión: {e}")

In [ ]:
def export_mongo_sample_to_csv(
    db_name: str,
    collection_name: str,
    out_csv: str = "dataset_sample.csv",
    sample_size: int = 100000,
    projection: dict | None = None,
):
    """
    Función ETL para extraer datos de MongoDB y convertirlos a CSV
    
    Args:
        db_name: Nombre de la base de datos
        collection_name: Nombre de la colección
        out_csv: Archivo CSV de salida
        sample_size: Número de documentos a extraer
        projection: Campos específicos a proyectar
    
    Returns:
        pandas.DataFrame: Datos extraídos
    """
    
    # ✅ Optimización: Usar caché local si ya existe
    if os.path.exists(out_csv):
        print(f"📂 Archivo {out_csv} ya existe, cargando desde caché...")
        df = pd.read_csv(out_csv)
        print(f"✅ Cargados {len(df):,} registros desde CSV existente")
        print(f"📊 Columnas disponibles: {len(df.columns)}")
        return df

    # 🔌 Conectar a MongoDB
    print(f"🔄 Conectando a MongoDB: {db_name}.{collection_name}")
    mongo_uri = os.getenv("CONECTION_DB")
    client = MongoClient(mongo_uri)
    col = client[db_name][collection_name]

    # 📝 Pipeline de agregación
    pipeline = [{"$sample": {"size": sample_size}}]
    if projection:
        pipeline.append({"$project": projection})
        print(f"🎯 Proyección aplicada: {list(projection.keys())}")

    # 📥 Extraer datos
    print(f"📊 Extrayendo muestra de {sample_size:,} documentos...")
    cursor = col.aggregate(pipeline)
    data = list(cursor)
    
    # 🔄 Transformar a DataFrame
    df = pd.json_normalize(data)
    
    # 💾 Guardar caché
    df.to_csv(out_csv, index=False)
    client.close()
    
    print(f"✅ Nueva muestra guardada en {out_csv}")
    print(f"📊 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    print(f"💾 Tamaño del archivo: {os.path.getsize(out_csv)/1024/1024:.2f} MB")
    
    return df

print("✅ Función de extracción definida correctamente")

In [ ]:
# 🚀 EJECUTAR EXTRACCIÓN DE DATOS
print("🔄 Iniciando proceso de extracción...")

df_raw = export_mongo_sample_to_csv(
    db_name="luminarias",
    collection_name="luminarias",
    out_csv="luminarias_dataset.csv",
    sample_size=5000
)

# 👀 Vista previa de los datos extraídos
print("\n📋 RESUMEN DE DATOS EXTRAÍDOS:")
print("="*50)
print(f"• Filas: {df_raw.shape[0]:,}")
print(f"• Columnas: {df_raw.shape[1]}")
print(f"• Memoria: {df_raw.memory_usage(deep=True).sum()/1024/1024:.2f} MB")

print("\n🔍 PRIMERAS 5 FILAS:")
display(df_raw.head())

print("\n📊 INFORMACIÓN DEL DATASET:")
print(df_raw.info())

## 🔄 FASE 2: TRANSFORM (Transformación)

Esta fase incluye limpieza, normalización, ingeniería de características y preparación de datos para ambos análisis (supervisado y no supervisado).

In [ ]:
# 🧹 LIMPIEZA Y PREPARACIÓN DE DATOS
print("🔄 Iniciando transformación de datos...")

# Copiar dataset para transformación
df = df_raw.copy()

print(f"📊 Dataset original: {df.shape}")

# ==========================================
# 1. ANÁLISIS DE CALIDAD DE DATOS
# ==========================================
print("\n🔍 ANÁLISIS DE CALIDAD DE DATOS:")
print("="*50)

# Valores nulos
null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df)) * 100
quality_report = pd.DataFrame({
    'Columna': null_counts.index,
    'Valores_Nulos': null_counts.values,
    'Porcentaje_Nulos': null_percentage.values
}).sort_values('Porcentaje_Nulos', ascending=False)

print("📋 Top 10 columnas con más valores nulos:")
display(quality_report.head(10))

# ==========================================
# 2. SELECCIÓN DE CARACTERÍSTICAS
# ==========================================
print("\n🎯 SELECCIÓN DE CARACTERÍSTICAS RELEVANTES:")

# Características para análisis supervisado y no supervisado
features_supervisado = [
    "potencia_watts",
    "altura_metros", 
    "consumo.actual_watts",
    "consumo.acumulado_kwh.mes",
    "consumo.acumulado_kwh.semana",
    "consumo.acumulado_kwh.dia",
    "sensores.luminosidad_lux",
    "sensores.temperatura_c",
    "sensores.humedad_pct",
    "sensores.movimiento",
    "conectividad.latencia_ms",
    "eficiencia.lumens_por_watt",
    "eficiencia.horas_funcionamiento_total",
    "eficiencia.vida_util_restante_pct",
    "estado_lampara"  # Variable objetivo
]

features_no_supervisado = [
    "potencia_watts",
    "altura_metros",
    "consumo.actual_watts", 
    "consumo.acumulado_kwh.mes",
    "sensores.luminosidad_lux",
    "sensores.temperatura_c",
    "sensores.humedad_pct",
    "conectividad.latencia_ms",
    "eficiencia.lumens_por_watt",
    "eficiencia.horas_funcionamiento_total",
    "eficiencia.vida_util_restante_pct"
]

# Verificar disponibilidad de características
available_features_sup = [col for col in features_supervisado if col in df.columns]
available_features_unsup = [col for col in features_no_supervisado if col in df.columns]

print(f"✅ Características disponibles para análisis supervisado: {len(available_features_sup)}/{len(features_supervisado)}")
print(f"✅ Características disponibles para análisis no supervisado: {len(available_features_unsup)}/{len(features_no_supervisado)}")

if len(available_features_sup) < len(features_supervisado):
    missing_sup = set(features_supervisado) - set(available_features_sup)
    print(f"⚠️ Características faltantes (supervisado): {missing_sup}")

if len(available_features_unsup) < len(features_no_supervisado):
    missing_unsup = set(features_no_supervisado) - set(available_features_unsup)
    print(f"⚠️ Características faltantes (no supervisado): {missing_unsup}")

In [ ]:
# ==========================================
# 3. LIMPIEZA DE DATOS
# ==========================================
print("\n🧹 PROCESO DE LIMPIEZA:")
print("="*30)

# Filtrar solo las características disponibles
df_supervisado = df[available_features_sup].copy() if available_features_sup else pd.DataFrame()
df_no_supervisado = df[available_features_unsup].copy() if available_features_unsup else pd.DataFrame()

if not df_supervisado.empty:
    print(f"📊 Dataset supervisado: {df_supervisado.shape}")
    
    # Eliminar filas con valores nulos en variables críticas
    antes_limpieza = len(df_supervisado)
    df_supervisado = df_supervisado.dropna()
    despues_limpieza = len(df_supervisado)
    
    print(f"🗑️ Filas eliminadas por valores nulos: {antes_limpieza - despues_limpieza:,}")
    print(f"✅ Filas restantes: {despues_limpieza:,}")

if not df_no_supervisado.empty:
    print(f"📊 Dataset no supervisado: {df_no_supervisado.shape}")
    
    # Eliminar filas con valores nulos
    antes_limpieza_unsup = len(df_no_supervisado)
    df_no_supervisado = df_no_supervisado.dropna()
    despues_limpieza_unsup = len(df_no_supervisado)
    
    print(f"🗑️ Filas eliminadas por valores nulos: {antes_limpieza_unsup - despues_limpieza_unsup:,}")
    print(f"✅ Filas restantes: {despues_limpieza_unsup:,}")

# ==========================================
# 4. INGENIERÍA DE CARACTERÍSTICAS
# ==========================================
print("\n🔧 INGENIERÍA DE CARACTERÍSTICAS:")
print("="*35)

if not df_supervisado.empty and 'estado_lampara' in df_supervisado.columns:
    # Análisis de la variable objetivo
    print("🎯 Variable objetivo (estado_lampara):")
    estado_counts = df_supervisado['estado_lampara'].value_counts()
    print(estado_counts)
    
    # Visualizar distribución de clases
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    estado_counts.plot(kind='bar', color=['green', 'orange', 'red'])
    plt.title('Distribución de Estados de Luminarias')
    plt.xlabel('Estado')
    plt.ylabel('Cantidad')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    plt.pie(estado_counts.values, labels=estado_counts.index, autopct='%1.1f%%', 
            colors=['green', 'orange', 'red'])
    plt.title('Proporción de Estados')
    plt.tight_layout()
    plt.show()

# Estadísticas descriptivas
if not df_no_supervisado.empty:
    print("\n📈 ESTADÍSTICAS DESCRIPTIVAS:")
    display(df_no_supervisado.describe())

## 📥 FASE 3: LOAD (Carga y Análisis)

### 🎯 ANÁLISIS SUPERVISADO - Random Forest Classification

In [ ]:
# 🎯 ANÁLISIS SUPERVISADO
print("🚀 Iniciando análisis supervisado...")

if not df_supervisado.empty and 'estado_lampara' in df_supervisado.columns:
    
    # ==========================================
    # PREPARACIÓN DE DATOS
    # ==========================================
    
    # Separar características y variable objetivo
    X = df_supervisado.drop('estado_lampara', axis=1)
    y = df_supervisado['estado_lampara']
    
    # Codificar variable objetivo si es necesario
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    print(f"✅ Características (X): {X.shape}")
    print(f"✅ Variable objetivo (y): {y.shape}")
    print(f"🏷️ Clases: {le.classes_}")
    
    # Escalar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # División train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    print(f"📊 Entrenamiento: {X_train.shape[0]} muestras")
    print(f"📊 Prueba: {X_test.shape[0]} muestras")
    
    # ==========================================
    # ENTRENAMIENTO DEL MODELO
    # ==========================================
    
    print("\n🌳 Entrenando Random Forest...")
    
    # Modelo Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    # Entrenar modelo
    rf_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = rf_model.predict(X_test)
    
    # ==========================================
    # EVALUACIÓN DEL MODELO
    # ==========================================
    
    print("\n📊 MÉTRICAS DE EVALUACIÓN:")
    print("="*30)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"🎯 Accuracy: {accuracy:.3f}")
    print(f"🎯 Precision: {precision:.3f}")
    print(f"🎯 Recall: {recall:.3f}")
    print(f"🎯 F1-Score: {f1:.3f}")
    
    print("\n📋 Reporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    
else:
    print("⚠️ No se puede realizar análisis supervisado: datos insuficientes o variable objetivo faltante")

In [ ]:
# ==========================================
# VISUALIZACIONES DEL ANÁLISIS SUPERVISADO
# ==========================================

if not df_supervisado.empty and 'estado_lampara' in df_supervisado.columns:
    
    print("\n📊 GENERANDO VISUALIZACIONES...")
    
    # 1. IMPORTANCIA DE VARIABLES
    print("📈 Gráfico 1: Importancia de Variables")
    
    # Obtener importancia de características
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=True)
    
    # Gráfico de importancia
    plt.figure(figsize=(12, 8))
    plt.barh(feature_importance['feature'], feature_importance['importance'])
    plt.title('Importancia de Variables (Random Forest)', fontsize=16, fontweight='bold')
    plt.xlabel('Score de Importancia', fontsize=12)
    plt.ylabel('Variables', fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    
    # Agregar valores en las barras
    for i, v in enumerate(feature_importance['importance']):
        plt.text(v + 0.001, i, f'{v:.3f}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('importancia_variables_etl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. MATRIZ DE CONFUSIÓN
    print("📈 Gráfico 2: Matriz de Confusión")
    
    # Calcular matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    
    # Gráfico de matriz de confusión
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=le.classes_, yticklabels=le.classes_,
                cbar_kws={'label': 'Cantidad de Predicciones'})
    plt.title('Matriz de Confusión', fontsize=16, fontweight='bold')
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('matriz_confusion_etl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Mostrar estadísticas de la matriz
    print("\\n📊 Análisis de la Matriz de Confusión:")
    for i, clase in enumerate(le.classes_):
        verdaderos_positivos = cm[i, i]
        total_reales = cm[i, :].sum()
        total_predichos = cm[:, i].sum()
        
        precision_clase = verdaderos_positivos / total_predichos if total_predichos > 0 else 0
        recall_clase = verdaderos_positivos / total_reales if total_reales > 0 else 0
        
        print(f"  • {clase}:")
        print(f"    - Predicciones correctas: {verdaderos_positivos}")
        print(f"    - Precision: {precision_clase:.3f}")
        print(f"    - Recall: {recall_clase:.3f}")
    
else:
    print("⚠️ No se pueden generar visualizaciones del análisis supervisado")

### 🔍 ANÁLISIS NO SUPERVISADO - K-Means Clustering & PCA

In [ ]:
# 🔍 ANÁLISIS NO SUPERVISADO
print("🚀 Iniciando análisis no supervisado...")

if not df_no_supervisado.empty:
    
    # ==========================================
    # PREPARACIÓN DE DATOS PARA CLUSTERING
    # ==========================================
    
    print(f"📊 Dataset para clustering: {df_no_supervisado.shape}")
    
    # Normalizar datos para clustering
    scaler_unsup = StandardScaler()
    X_scaled_unsup = scaler_unsup.fit_transform(df_no_supervisado)
    
    print("✅ Datos normalizados para clustering")
    
    # ==========================================
    # DETERMINACIÓN DEL NÚMERO ÓPTIMO DE CLUSTERS
    # ==========================================
    
    print("\\n🔄 Determinando número óptimo de clusters...")
    
    # Método del codo
    wcss = []
    silhouette_scores = []
    k_range = range(2, 8)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled_unsup)
        wcss.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled_unsup, kmeans.labels_))
    
    # ==========================================
    # VISUALIZACIÓN: MÉTODO DEL CODO Y SILHOUETTE
    # ==========================================
    
    print("📈 Gráfico 3: Método del Codo y Puntuación de Silhouette")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Método del codo
    ax1.plot(k_range, wcss, 'bo-', linewidth=2, markersize=8)
    ax1.set_title('Método del Codo', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Número de clusters', fontsize=12)
    ax1.set_ylabel('WCSS', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Silhouette score
    ax2.plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
    ax2.set_title('Puntuación de Silhouette por Número de Clusters', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Número de clusters', fontsize=12)
    ax2.set_ylabel('Puntuación de Silhouette', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('metodo_codo_silhouette_etl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Encontrar k óptimo
    optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
    print(f"\\n🎯 Número óptimo de clusters: {optimal_k}")
    print(f"🎯 Mejor Silhouette Score: {max(silhouette_scores):.3f}")
    
else:
    print("⚠️ No se puede realizar análisis no supervisado: datos insuficientes")

In [ ]:
# ==========================================
# APLICAR K-MEANS CON K ÓPTIMO
# ==========================================

if not df_no_supervisado.empty:
    
    print(f"\\n🎯 Aplicando K-Means con k={optimal_k}...")
    
    # Modelo K-Means final
    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    cluster_labels = kmeans_final.fit_predict(X_scaled_unsup)
    
    # Calcular silhouette score final
    silhouette_final = silhouette_score(X_scaled_unsup, cluster_labels)
    print(f"✅ Silhouette Score final: {silhouette_final:.3f}")
    
    # ==========================================
    # REDUCCIÓN DE DIMENSIONALIDAD CON PCA
    # ==========================================
    
    print("\\n🔄 Aplicando PCA para visualización...")
    
    # PCA a 2 componentes para visualización
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_scaled_unsup)
    
    # Varianza explicada
    varianza_explicada = pca.explained_variance_ratio_
    print(f"✅ Varianza explicada por PC1: {varianza_explicada[0]:.3f}")
    print(f"✅ Varianza explicada por PC2: {varianza_explicada[1]:.3f}")
    print(f"✅ Varianza total explicada: {sum(varianza_explicada):.3f}")
    
    # ==========================================
    # VISUALIZACIÓN: CLUSTERS EN ESPACIO PCA
    # ==========================================
    
    print("📈 Gráfico 4: Visualización de Clusters en Espacio PCA")
    
    plt.figure(figsize=(12, 8))
    
    # Colores para cada cluster
    colors = plt.cm.Set1(np.linspace(0, 1, optimal_k))
    
    for i in range(optimal_k):
        mask = cluster_labels == i
        plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                   c=[colors[i]], label=f'Cluster {i}', 
                   alpha=0.7, s=50)
    
    # Centroides en espacio PCA
    centroids_pca = pca.transform(kmeans_final.cluster_centers_)
    plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
               c='black', marker='x', s=200, linewidths=3, label='Centroides')
    
    plt.title('Visualización de Clusters de Luminarias (PCA)', fontsize=16, fontweight='bold')
    plt.xlabel(f'Componente Principal 1 ({varianza_explicada[0]:.1%} varianza)', fontsize=12)
    plt.ylabel(f'Componente Principal 2 ({varianza_explicada[1]:.1%} varianza)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('clusters_pca_etl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # ==========================================
    # ANÁLISIS DE CLUSTERS
    # ==========================================
    
    print("\\n📊 ANÁLISIS DE CARACTERÍSTICAS POR CLUSTER:")
    print("="*50)
    
    # Agregar labels de cluster al dataframe
    df_clusters = df_no_supervisado.copy()
    df_clusters['cluster'] = cluster_labels
    
    # Estadísticas por cluster
    cluster_stats = df_clusters.groupby('cluster').agg(['mean', 'std', 'count'])
    
    print("📈 Características promedio por cluster:")
    display(df_clusters.groupby('cluster').mean().round(2))
    
    # Distribución de clusters
    cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
    print(f"\\n📊 Distribución de luminarias por cluster:")
    for i, count in cluster_counts.items():
        percentage = (count / len(cluster_labels)) * 100
        print(f"  • Cluster {i}: {count:,} luminarias ({percentage:.1f}%)")

else:
    print("⚠️ No se pueden generar visualizaciones del análisis no supervisado")

In [ ]:
# ==========================================
# VISUALIZACIÓN: DISTRIBUCIÓN DE CARACTERÍSTICAS POR CLUSTER
# ==========================================

if not df_no_supervisado.empty and len(df_clusters.columns) > 1:
    
    print("📈 Gráfico 5: Distribución de Características por Cluster")
    
    # Seleccionar las variables más importantes para visualizar
    variables_importantes = [
        'potencia_watts', 'altura_metros', 'consumo.actual_watts',
        'sensores.luminosidad_lux', 'sensores.temperatura_c', 'sensores.humedad_pct',
        'eficiencia.lumens_por_watt', 'eficiencia.horas_funcionamiento_total',
        'eficiencia.vida_util_restante_pct', 'conectividad.latencia_ms'
    ]
    
    # Filtrar variables que existen en el dataset
    variables_disponibles = [var for var in variables_importantes if var in df_clusters.columns]
    
    if len(variables_disponibles) >= 4:
        # Crear subplots para boxplots
        n_vars = len(variables_disponibles)
        n_cols = 4
        n_rows = (n_vars + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, var in enumerate(variables_disponibles):
            row = i // n_cols
            col = i % n_cols
            
            ax = axes[row, col] if n_rows > 1 else axes[col]
            
            # Boxplot por cluster
            df_clusters.boxplot(column=var, by='cluster', ax=ax)
            ax.set_title(var.replace('.', '.\\n'))
            ax.set_xlabel('Cluster')
            ax.set_ylabel(var.split('.')[-1])
            
        # Ocultar subplots vacíos
        for i in range(len(variables_disponibles), n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            axes[row, col].set_visible(False)
        
        plt.suptitle('Distribución de Características por Cluster', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig('distribucion_clusters_etl.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    # ==========================================
    # INTERPRETACIÓN DE CLUSTERS
    # ==========================================
    
    print("\\n🎯 INTERPRETACIÓN DE CLUSTERS:")
    print("="*35)
    
    cluster_means = df_clusters.groupby('cluster').mean()
    
    for cluster_id in range(optimal_k):
        print(f"\\n🔍 **Cluster {cluster_id}:**")
        cluster_data = cluster_means.loc[cluster_id]
        
        # Identificar características destacadas
        caracteristicas_altas = []
        caracteristicas_bajas = []
        
        for var in variables_disponibles:
            if var in cluster_data.index:
                valor = cluster_data[var]
                promedio_general = df_clusters[var].mean()
                std_general = df_clusters[var].std()
                
                if valor > promedio_general + 0.5 * std_general:
                    caracteristicas_altas.append(f"{var}: {valor:.2f}")
                elif valor < promedio_general - 0.5 * std_general:
                    caracteristicas_bajas.append(f"{var}: {valor:.2f}")
        
        if caracteristicas_altas:
            print(f"   📈 Características altas: {', '.join(caracteristicas_altas[:3])}")
        if caracteristicas_bajas:
            print(f"   📉 Características bajas: {', '.join(caracteristicas_bajas[:3])}")
        
        # Contar luminarias en este cluster
        count = cluster_counts[cluster_id]
        percentage = (count / len(cluster_labels)) * 100
        print(f"   📊 Luminarias: {count:,} ({percentage:.1f}%)")

else:
    print("⚠️ No se puede generar análisis detallado de clusters")

## 📊 RESULTADOS Y CONCLUSIONES DEL ETL

### 🎯 Resumen del Proceso ETL

El proceso ETL implementado ha demostrado ser efectivo para la preparación y análisis de datos de luminarias urbanas, integrando exitosamente metodologías supervisadas y no supervisadas.

In [ ]:
# ==========================================
# RESUMEN FINAL Y RECOMENDACIONES
# ==========================================

print("📋 RESUMEN EJECUTIVO DEL PROCESO ETL")
print("="*50)

# Resumen de datos procesados
print(f"\\n📊 DATOS PROCESADOS:")
if not df_raw.empty:
    print(f"  • Registros extraídos: {len(df_raw):,}")
    print(f"  • Columnas originales: {df_raw.shape[1]}")

if not df_supervisado.empty:
    print(f"  • Registros para análisis supervisado: {len(df_supervisado):,}")
    print(f"  • Características supervisadas: {len(df_supervisado.columns)-1}")

if not df_no_supervisado.empty:
    print(f"  • Registros para análisis no supervisado: {len(df_no_supervisado):,}")
    print(f"  • Características no supervisadas: {len(df_no_supervisado.columns)}")

# Resumen de resultados
print(f"\\n🎯 RESULTADOS OBTENIDOS:")

if 'accuracy' in locals():
    print(f"  • Accuracy del modelo supervisado: {accuracy:.1%}")
    print(f"  • Precisión promedio: {precision:.1%}")

if 'silhouette_final' in locals():
    print(f"  • Silhouette Score del clustering: {silhouette_final:.3f}")
    print(f"  • Número óptimo de clusters: {optimal_k}")

# Archivos generados
print(f"\\n📁 ARCHIVOS GENERADOS:")
archivos_generados = [
    "luminarias_dataset.csv",
    "importancia_variables_etl.png", 
    "matriz_confusion_etl.png",
    "metodo_codo_silhouette_etl.png",
    "clusters_pca_etl.png",
    "distribucion_clusters_etl.png"
]

for archivo in archivos_generados:
    if os.path.exists(archivo):
        size_mb = os.path.getsize(archivo) / 1024 / 1024
        print(f"  ✅ {archivo} ({size_mb:.2f} MB)")
    else:
        print(f"  ⚠️ {archivo} (no encontrado)")

# ==========================================
# RECOMENDACIONES Y PRÓXIMOS PASOS
# ==========================================

print(f"\\n💡 RECOMENDACIONES:")
print("="*20)

print("\\n🔍 **Para Análisis Supervisado:**")
print("  • Considerar recolectar más datos etiquetados")
print("  • Explorar técnicas de balanceo de clases")
print("  • Implementar validación cruzada")
print("  • Probar algoritmos alternativos (XGBoost, SVM)")

print("\\n🎯 **Para Análisis No Supervisado:**")
print("  • Validar clusters con conocimiento del dominio")
print("  • Considerar algoritmos de clustering alternativos (DBSCAN, Hierarchical)")
print("  • Explorar más componentes principales")
print("  • Implementar análisis de outliers")

print("\\n🔄 **Para el Proceso ETL:**")
print("  • Automatizar el pipeline completo")
print("  • Implementar monitoreo de calidad de datos")
print("  • Establecer actualizaciones incrementales")
print("  • Crear alertas para anomalías en los datos")

print("\\n🎉 **CONCLUSIÓN:**")
print("El proceso ETL ha proporcionado una base sólida para el análisis")
print("de luminarias urbanas, identificando patrones importantes y")
print("estableciendo métricas de rendimiento para futuros modelos.")
print("\\n✅ Proceso ETL completado exitosamente!")